In [1]:
import pandas as pd
import re

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import eli5
import numpy as np
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

2022-08-27 17:06:51.753172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-27 17:06:51.753200: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/annavlasova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import json

with open('../data/full_docs.json', 'r+') as fp:
    full_docs1 = json.load(fp)

In [3]:
files = list(full_docs1.keys())

In [4]:
df_list = []
for i in range(len(files)):
    df1 = pd.DataFrame(full_docs1[files[i]])
#    df_list.append(df1)

In [5]:
#df = pd.concat(df_list)
df = df1

In [6]:
df

,data,text,target,few_texts,few_targets
0,"1. {2}Настоящие Правила устанавливают цели, ус...","Настоящие Правила устанавливают цели, условия ...",2,"[1. Настоящие Правила устанавливают цели, усло...",[2]
1,{7}Сведения о субсидиях размещаются на едином ...,Сведения о субсидиях размещаются на едином пор...,7,[Сведения о субсидиях размещаются на едином по...,[7]
2,2. {2} Субсидии предоставляются по результатам...,Субсидии предоставляются по результатам провед...,2,[2. Субсидии предоставляются по результатам п...,"[2, 2]"
3,"3. {1}Понятия, используемые в настоящих Правил...","Понятия, используемые в настоящих Правилах, оз...",1,"[3. Понятия, используемые в настоящих Правилах...",[1]
4,"{1}""организация воздушного транспорта"" - росси...","""организация воздушного транспорта"" - российск...",1,"[""организация воздушного транспорта"" - российс...",[1]
...,...,...,...,...,...
217,16. Правительства Российской Федерации от 27 ...,16. Правительства Российской Федерации от 27 ...,0,[16. Правительства Российской Федерации от 27...,[0]
218,"17. и изменений, которые вносятся в акты Пра...","17. и изменений, которые вносятся в акты Пра...",0,"[17. и изменений, которые вносятся в акты Пр...",[0]
219,18. Правительства Российской Федерации от 24 ...,18. Правительства Российской Федерации от 24 ...,0,[18. Правительства Российской Федерации от 24...,[0]
220,19. Правительства Российской Федерации от 29 ...,19. Правительства Российской Федерации от 29 ...,0,[19. Правительства Российской Федерации от 29...,[0]


In [7]:
df_list = []
for i in range(1, df.shape[0]+1):
    try:
        df2 = df.iloc[i-1:i].explode(['few_texts', 'few_targets'], ignore_index=True)
        df_list.append(df2)
    except:
        print(i)

In [8]:
df = pd.concat(df_list)

In [9]:
df['ref_text'] = df['few_texts'].apply(lambda x: re.sub('[\W\d]+', ' ', x.lower()))

In [10]:
df.fillna(0, inplace = True)

In [11]:
X = df['ref_text']

In [12]:
y = df['few_targets']

In [13]:
text_transformer = TfidfVectorizer(stop_words=russian_stopwords, 
                                   ngram_range=(1, 2), lowercase=True, max_features=1000)

In [14]:
X_text = text_transformer.fit_transform(X)

In [15]:
#logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)

In [16]:
#%%time
#logit.fit(X_text, y)

In [17]:
import pickle
filename = '../models/log_reg_model.pickle'

#pickle.dump(logit, open(filename, 'wb'))

In [18]:
model = pickle.load(open(filename, 'rb'))

In [20]:
y_pred = model.predict(X_text)
y_proba = model.predict_proba(X_text)

In [24]:
s = []
for i in range(len(y_pred)):
    k = y_pred[i]
    s.append(y_proba[i, k])
    
df_res = pd.DataFrame()
df_res['proba'] = s
df_res['preds'] = y_pred

df_res = df_res.groupby('preds')['proba'].mean().reset_index()

df_dump = pd.DataFrame()
df_dump['preds'] = [x for x in range(1, 40)]
df_dump = df_dump.merge(df_res, on = 'preds', how = 'left')
print(f"Полнота документа: {round(df_res.shape[0]/39*100, 2)}%")

df_dump.fillna('отсутствует в документе', inplace = True)
print(list(df_dump['proba']))
print(list(df_res['preds']))
print(set(y))

Полнота документа: 48.72%
[0.40910837570910197, 0.4667129169781693, 'отсутствует в документе', 0.3973957116833611, 'отсутствует в документе', 'отсутствует в документе', 0.5503671969653923, 'отсутствует в документе', 'отсутствует в документе', 0.3889165598639848, 0.6180301098823403, 0.46552733134332125, 'отсутствует в документе', 'отсутствует в документе', 0.36055202731612196, 'отсутствует в документе', 'отсутствует в документе', 'отсутствует в документе', 0.6237854909931151, 'отсутствует в документе', 0.5858393529110301, 0.4388867614350225, 'отсутствует в документе', 0.5122615757790187, 'отсутствует в документе', 'отсутствует в документе', 0.2668592970021411, 0.27874458226797244, 'отсутствует в документе', 0.6335366994120774, 'отсутствует в документе', 'отсутствует в документе', 'отсутствует в документе', 'отсутствует в документе', 0.4432741787370886, 0.2747716127352877, 0.25267163980803825, 'отсутствует в документе', 'отсутствует в документе']
[0, 1, 2, 4, 7, 10, 11, 12, 15, 19, 21, 2